In [2]:
%ls
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import cm
from matplotlib import gridspec
import numpy as np
import pandas as pd
import mdtraj as md
import subprocess, collections
import glob, tqdm, os
import xvg_tools
rc('font', weight='bold')

plots/              REF_L_RUN1_lam.npy   REF_RL_RUN1_feb.npy
qsub.sh             REF_L_RUN2_feb.npy   REF_RL_RUN1_inc.npy
REF_L/              REF_L_RUN2_inc.npy   REF_RL_RUN1_lam.npy
REF_L_RUN0_feb.npy  REF_L_RUN2_lam.npy   REF_RL_RUN2_feb.npy
REF_L_RUN0_inc.npy  REF_RL/              REF_RL_RUN2_inc.npy
REF_L_RUN0_lam.npy  REF_RL_RUN0_feb.npy  REF_RL_RUN2_lam.npy
REF_L_RUN1_feb.npy  REF_RL_RUN0_inc.npy  scrape.ipynb
REF_L_RUN1_inc.npy  REF_RL_RUN0_lam.npy


In [ ]:
# TODO
# Replace structure in dG1 with first frame of centered trajectory file
# Replace distances in dG4 AND change MBAR function to compute potential for THREE restraints

In [50]:
dataset = 'REF'
prefix = '.'


for run in range(len(glob.glob(f'{prefix}/{dataset}_RL/RUN*'))):
    if not os.path.exists(f'results/scraped_data/{dataset}_RL_RUN{run}_feb.npy') or not os.path.exists(
      f'scraped_data/{dataset}_L_RUN{run}_feb.npy'):
        continue # skip if we don't have scraped free energies for dG2 and dG3
    
# Compute dG2 and dG3 
    RL_increments = np.load(f'scraped_data/{dataset}_RL_RUN{run}_inc.npy')
    RL_fe = np.load(f'scraped_data/{dataset}_RL_RUN{run}_feb.npy')[:,-1]
    L_increments = np.load(f'scraped_data/{dataset}_L_RUN{run}_inc.npy')
    L_fe = np.load(f'scraped_data/{dataset}_L_RUN{run}_feb.npy')[:,-1]
    
    for x,i in enumerate(L_increments):
        if i < increment_threshold:
            break
    #print(np.shape(L_fe), x, np.average(L_fe[x:]), L_fe[-1]) # < --- decide which to use
    dG2 = np.average(L_fe[x:]
    
    for x,i in enumerate(RL_increments):
        if i < increment_threshold:
            last_frames = x*10 # un-stride for later
            break
    #print(np.shape(RL_fe), x, np.average(RL_fe[x:]), RL_fe[-1])
    dG3 = -np.average(RL_fe[x:])
    
# Compute dG1
    itpfile = f'{prefix}/{dataset}_RL/RUN{run}/LIG_res.itp'
    fin = open(itpfile, 'r')
    lines = fin.readlines()
    fin.close()

    rest_lines = [line for line in lines if (line[0] != ';' and line[0] != '[')]
    rest_indices = []
    for line in rest_lines:
        fields = line.strip().split()
        rest_indices.append( int(fields[0]) )

    structure = md.load(f'{prefix}/{dataset}_RL/RUN{run}/npt.gro') # <-- replace with traj
    x1 = structure.xyz[0][rest_indices[0]-1]
    x2 = structure.xyz[0][rest_indices[1]-1]
    x3 = structure.xyz[0][rest_indices[2]-1]
    G1 = dG_harmonic_rest_triple(x1,x2,x3)
    
# Compute dG4
    time_in_ps, states, energies = xvg_tools.get_dhdl(f'{prefix}/{dataset}_RL/RUN{run}/dhdl.xvg')
    # should we get just last few nanoseconds for estimating free energy?
    time_in_ps = time_in_ps[::100] # stride to match xtc save frequency
    states = states[::100]
    energies = energies[::100]

    traj = md.load(f'{prefix}/{dataset}_RL/RUN{run}/traj.xtc',
      top=f'{prefix}/{dataset}_RL/RUN{run}/npt.gro') # <-- get distances from here?
    
    dG4, sigma_dG4 = dG_restraint_MBAR(states, energies, distances, kvalue=800, # kJ/mol/nm^2
        r0=r0, temperature=298.15, verbose=False) # <-- will fix u_kn for multi-restraints
    
    FEB = dG1 + dG2 + dG3 + dG4

(2655,) 398 155.30147557820118 148.90857
(13909,) 6864 168.41655517104329 168.45462
[6, 35, 23]


NameError: name 'xvg_tools' is not defined

In [38]:
def dG_harmonic_rest_triple(x1,x2,x3, k=800.0, L=1.184048, verbose=False):

    """Returns the free energy of turning on a triple harmonic restraint in units RT, 
    for a rigid nonlinear molecule restrained at anchors x1, x2, and x3
    INPUTS
    x1, x2, x3 - the coordinates of the three restraint positions as 3-element np.array(), each in units nm
    PARAMETERS
    k      - the force constant, in kJ/mol/nm^2  (Default: 800.0 kJ/mol/nm^2)
    L      - the length of the cubic box (Default: 1.184048 nm ) 
                V0 = 1660.0                   # in Angstrom^3 is the standard volume
                L0 = ((1660.0)**(1/3)) * 0.1  # converted to nm
                L0 = 1.1840481475428983 nm
    """

    RT      = 2.479           # in kJ/mol at 298 K


    # Calculate distance between particles 1 and 2
    d = np.sqrt( np.dot(x2-x1,x2-x1) )
    #d = md_compute_distances(structure, [indices[0],indices[1]])[0]    
    # Calculate the altitude (height), c, of triangle where p1-p2 is the base
    ### 1. First calculate the area of the triangle as A = (1/2)||v \cross w||
    v, w = x2-x1, x3-x1
    area = 0.5*np.linalg.norm( np.cross(v,w) )
    ### 2. Then since A = 1/2 * base * height, the height is c = 2*area/base
    c = 2.0 * area / np.linalg.norm(v)
        
    # Calculate e, the projection of x3-x1 = w in the x2-x1 = v direction
    unit_vec_along_x12 = v / d
    e = np.abs(np.dot(w, unit_vec_along_x12))
        
    if verbose:
        print('Distance betweeen x1 and x2, \td =', d, 'nm')
        print('Height of x3 from triangular base x1-x2, \tc =', c, 'nm')
        print('Projection of x3-x1 in the x2-x1 direction, \te =', e, 'nm')

    kc_coeff = 1.0 + (e/d)**2.0
    if verbose:
        print('kc_coeff', kc_coeff)
    kp1_coeff = 1.0 + (c**2 + e**2)/(d**2)
    if verbose:
        print('kp1_coeff', kp1_coeff)

    '''
        theory_dG_in_kT = -1.0*( 3.0/2.0*np.log(2.0*np.pi*ee.RT/(3.0*ee.k_values[1:]))        \ # translational
                                + 1.0/2.0*np.log(2.0*np.pi*ee.RT/(2.0*ee.k_values[1:]))       \ # rot about d
                                + 1.0/2.0*np.log(2.0*np.pi*ee.RT/(kc_coeff*ee.k_values[1:]))  \ # rot about c
                                + 1.0/2.0*np.log(2.0*np.pi*ee.RT/(kp1_coeff*ee.k_values[1:])) \ # rot out of page
                                - np.log( ee.L**3 * 8.0 * (np.pi**2) * (ee.d)**2 * ee.c  ) )
    '''

    theory_dG_in_kT = -1.0*( 3.0/2.0*np.log(2.0*np.pi*RT/(3.0*k))            \
                                + 1.0/2.0*np.log(2.0*np.pi*RT/k)             \
                                + 1.0/2.0*np.log(2.0*np.pi*RT/(kc_coeff*k))  \
                                + 1.0/2.0*np.log(2.0*np.pi*RT/(kp1_coeff*k)) \
                                - np.log( L**3 * 8.0 * (np.pi**2) * d**2 * c  ) )
    return theory_dG_in_kT

In [47]:
def dG_restraint_MBAR(states, energies, distances,
                         kvalue=800.0, r0=0.4, temperature=300., verbose=False):
    """Use MBAR to estimate the free energy vs. lambda.
    N is the number of samples
    K is the number of thermodynamic states

    INPUTS
    states    - state indices in a numpy array of shape (N,)
    energies  - a numpy array of shape (N, K) with the dhdl info
    distances - restraint distances numpy array of shape (N,)

    PARAMETERS
    kvalue      - the harmonic force constant in kJ/nm^2 (Default: 400.0)
    r0          - the restraint distance umbrella center, in nm.
    temperature - in K (Default: 300.0)
    verbose     - print verbose output

    OUTPUT
    """

    ###########################
    # Main

    # Physical constants (in kJ)
    kB = 1.381e-23 * 6.022e23 / 1000.0 # Boltzmann constant in kJ/mol/K

    # In addition to the given K thermo ensembles with indices 0,..K-1,
    # there is one more -- the *unbiased* ensemble with no harmonic restraint.
    #  -- let's make it state index K
    K = energies.shape[1] + 1
    unbiased_state_index = K - 1


    # maximum number of snapshots/simulation:
    N_max = energies.shape[0]
    ### TO DO in the future collect *all* run and clone energies and flatten

    T_k = np.ones(K,float)*temperature # inital temperatures are all equal
    beta = 1.0 / (kB * temperature) # inverse temperature of simulations (in 1/(kJ/mol))
    if verbose:
        print('beta',  beta)

    # Allocate storage for simulation data
    N_k = np.zeros([K], np.int32) # N_k[k] is the number of snapshots from umbrella simulation k
    # rvalues[k] is the spring center location (in nm) for umbrella simulation k
    x_kn = np.zeros([K,N_max], np.float64) # x_kn[k,n] is the Val122_CA-TRP_CA distance (in nm) for snapshot n from umbrella simulation k
    u_kn = np.zeros([K,N_max], np.float64) # u_kn[k,n] is the reduced potential energy without umbrella restraints of snapshot n of umbrella simulation k
    g_k = np.zeros([K],np.float32);


    ### To do MBAR, we need to convert to data to u_kn format
    if verbose:
        print('np.argsort(states)', np.argsort(states))

    Ind = np.argsort(states)
    energies_sorted = energies[Ind,:]
    states_sorted = states[Ind]
    distances_sorted = distances[Ind]
    for k in range(K-1):

        # Count how many snapshots belong to each k
        N_k[k] = np.where(states_sorted == k, 1, 0).sum()

        # fill the energies
        u_kn[k, :] = energies_sorted[:, k]

    # for the last (unbiased) ensemble (with no samples), subtract the harmonic potential from the
    # state index 0 (totally coupled) eneries
    u_kn[K-1, :] = u_kn[0, :] - beta * (kvalue/2.0) * (distances_sorted - r0)**2
    if verbose:
        print('u_kn', u_kn)
        print('N_k', N_k)

    # Initialize MBAR.
    if verbose:
        print('Running MBAR...')
    mbar = pymbar.MBAR(u_kn, N_k, verbose=verbose)  #, maximum_iterations=100000, initialize='BAR')  

    # MBAR(u_kn, N_k, maximum_iterations=10000, relative_tolerance=1e-07, verbose=False, initial_f_k=None, solver_protocol=None, initialize='zeros', x_kindices=None, **kwargs))

    # Set zero of u_kn -- this is arbitrary.
    u_kn -= u_kn.min()

    results = mbar.getFreeEnergyDifferences()
    Deltaf_ij, dDeltaf_ij = results[0], results[1]
    if verbose:
        print('Deltaf_ij, dDeltaf_ij', Deltaf_ij, dDeltaf_ij)

    df, sigma_df = np.zeros(K), np.zeros(K)
    for i in range(K-1):
        #  print('Deltaf_%d,%d = '%(i,i+1), Deltaf_ij[i,i+1], '+/-', dDeltaf_ij[i,i+1])
        df[i+1] = df[i] + Deltaf_ij[i,i+1]
        sigma_df[i+1] = dDeltaf_ij[0,i+1]

    dG_rest       = df[0] - df[-1]      # THIS should be the same as +f[-1] of the MBAR object!
    sigma_dG_rest = sigma_df[-1]

    if verbose:
        print('Delta f (norest, lam=1 -> rest, lam=1) =', df[0] - df[-1])
        print('dDelta f (norest, lam=1 -> rest, lam=1) =', sigma_df[-1])

    return dG_rest, sigma_dG_rest


In [49]:
import mdtraj as md
traj = md.load('REF_RL/RUN0/traj_comp.xtc',top='REF_RL/RUN0/xtc.gro')
print([a.index for a in traj.topology.atoms if a.residue.name == 'LIG'])
# NOTE that if you want LIG atoms you have to specify a.index < ~200
# bc some other residues got re-named :\

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564, 2565, 2566, 2567, 2568, 2569, 2570, 2571, 2572, 2573, 2574, 2694, 2695, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710]
